In [1]:
# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
import math

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.14.0-dev20230525


https://medium.com/@andeyharsha15/deep-neural-networks-for-solving-differential-equations-in-finance-da662ef0681

# Black Scholes Equation
$
{\frac {\partial V}{\partial t}}+{\frac {1}{2}}\sigma ^{2}S^{2}{\frac {\partial ^{2}V}{\partial S^{2}}}+rS{\frac {\partial V}{\partial S}}-rV=0
$

In [8]:
S = tf.Variable([80.0], dtype=tf.float32)
t = tf.Variable([0.0], dtype=tf.float32)
sigma = tf.Variable([0.3], dtype=tf.float32)
r = tf.Variable([0.05], dtype=tf.float32)
K = tf.constant([70.0], dtype=tf.float32)
T = tf.constant([1.0], dtype=tf.float32)
t2m = T - t
d1 = (tf.math.log(S / K) + (r + 0.5 * sigma**2) * t2m) / (sigma * tf.sqrt(t2m))
d2 = d1 - sigma * tf.sqrt(t2m)
N0 = lambda value: 0.5 * (1 + tf.math.erf(value / tf.sqrt(2.0))) # type: ignore
Nd1 = N0(d1)
Nd2 = N0(d2)
C = S * Nd1 - K * Nd2 * tf.exp(-r * t2m)

print("Option Price:", C[0]) #17.01496

Option Price: tf.Tensor(17.014957, shape=(), dtype=float32)


In [6]:
with tf.GradientTape() as tape:
    tape.watch([t, S, sigma, r])
    theta = tape.gradient(C, t)[0]
    delta = tape.gradient(C, S)[0]
    gamma = tape.gradient(delta, S)[0]
    vega = tape.gradient(C, sigma)[0]
    rho = tape.gradient(C, r)[0]

print(f"Theta: {theta.numpy():.4f}" if theta is not None else "Theta: N/A")
print(f"Delta: {delta.numpy():.4f}" if delta is not None else "Delta: N/A")
print(f"Vega: {vega.numpy():.4f}" if vega is not None else "Vega: N/A")
print(f"Rho: {rho.numpy():.4f}" if rho is not None else "Rho: N/A")

result = (-theta + 0.5 * sigma**2 * S**2 * 0 + r * S * delta - r * C).numpy()
print(f"Result: {result:.4f}")

TypeError: 'NoneType' object is not subscriptable